<a href="https://colab.research.google.com/github/killianraes/Machine_Learning_course_UGent_D012554_kaggle/blob/master/Eye_blinking_prediction/workpage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import pandas as pd


<img src="https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/header.png" alt="drawing"/>


A multi-channel electroencephalography (EEG) system enables a broad range of applications including neurotherapy, biofeedback, and brain computer interfacing. The dataset you will analyse is created with the [Emotiv EPOC+](https://www.emotiv.com/product/emotiv-epoc-14-channel-mobile-eeg).  

It has 14 EEG channels with names based on the International 10-20 locations: AF3, F7, F3, FC5, T7, P7, O1, O2, P8, T8, FC6, F4, F8, AF4:

<br/>
<br/>
<center>
<img src="https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/EEG.png" alt="drawing" width="200"/>
<center/>
<br/>
<br/>


All data is from one continuous EEG measurement with the Emotiv EEG Neuroheadset. 

The experiment was conducted on one person only. The duration of the measurement was around 117 seconds.

From the paper:

> *The experiment was carried out in a quiet room. During
the experiment, the proband was being videotaped. To prevent
artifacts, the proband was not aware of the exact start time
of the measurement. Instead, he was told to sit relaxed, look
straight to the camera, and change the eye state at free will.
Only additional constraint was that, accumulated over the
entire session, the duration of both eye states should be about
the same and that the individual intervals should vary greatly
in length (from eye blinking to longer stretches)...*

The eye state was detected via a camera during the EEG measurement and later added manually to the file after analyzing the video frames. 

A label '1' indicates the eye-closed and '0' the eye-open state.

(*Source: Oliver Roesler, Stuttgart, Germany*)

Let's load the train and test set:

In [0]:
trainset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_train.csv")

testset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_test.csv")

sample_submission = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/sample_submission.csv")


You will fit a model on the trainset and make predictions on the testset. 

To submit these predictions to Kaggle you need to write a .csv file with two columns: 
- `index` that matches the `index` column in the test set.
- `label` which is your prediction.

Here is an example predictions file for Kaggle:

In [17]:
sample_submission.head(10)

,index,label
0,0,0.168801
1,1,0.124169
2,2,0.947757
3,3,0.069585
4,4,0.635325
5,5,0.659027
6,6,0.653697
7,7,0.850030
8,8,0.160489
9,9,0.843272


In [0]:
#Make copy's of data, in case something goes wrong
data_train = trainset.copy()
data_test = testset.copy()

In [19]:
#Study training data
print(data_train.head())
print(data_train.describe())
print(data_train.label.value_counts())

       AF3       F7       F3      FC5  ...       F4       F8      AF4  label
0  4299.49  3997.44  4277.95  4116.92  ...  4278.97  4600.00  4369.23      1
1  4302.05  3985.64  4261.03  4129.74  ...  4283.08  4607.18  4358.46      0
2  4321.03  4015.90  4265.13  4122.56  ...  4286.15  4608.21  4371.79      0
3  4408.21  4104.10  4380.00  4232.31  ...  4388.21  4715.90  4464.10      0
4  4347.18  3975.38  4266.67  4102.56  ...  4313.33  4664.10  4411.79      1

[5 rows x 15 columns]
               AF3           F7  ...          AF4        label
count  2000.000000  2000.000000  ...  2000.000000  2000.000000
mean   4300.157125  4009.273150  ...  4359.852780     0.450500
std      36.361719    29.853264  ...    37.074555     0.497668
min    4197.950000  3905.640000  ...  4225.640000     0.000000
25%    4280.510000  3990.770000  ...  4342.050000     0.000000
50%    4293.330000  4006.150000  ...  4354.360000     0.000000
75%    4309.740000  4023.590000  ...  4371.790000     1.000000
max    4497

In [20]:
#Preprocess training data
from sklearn.preprocessing import StandardScaler

X_train = data_train.copy()
y_train = X_train.pop('label')

feature_scaler = StandardScaler()
feature_scaler.fit(X_train)

X_train = pd.DataFrame(feature_scaler.transform(X_train))
X_train.columns = data_train.columns[:-1]

X_train.head()


,AF3,F7,F3,FC5,T7,P7,O1,02,P8,T8,FC6,F4,F8,AF4
0,-0.018351,-0.396476,0.677912,-0.277047,0.733699,0.167400,1.330668,0.454721,0.093570,-0.068336,0.418469,0.026714,-0.156365,0.252992
1,0.052070,-0.791842,-0.136210,0.346482,-0.434286,-0.259658,0.007585,-1.609301,-0.482085,-0.276922,-0.232997,0.235973,0.060820,-0.037576
2,0.574178,0.222036,0.061066,-0.002733,-0.496011,-0.373355,0.031954,-0.716795,-0.482085,-0.381214,-1.894254,0.392280,0.091976,0.322059
3,2.972354,3.177226,5.588146,5.335202,6.480134,7.221060,4.638614,6.311759,6.199439,6.193291,4.834001,5.588622,3.349453,2.812530
4,1.293521,-1.135609,0.135164,-0.975478,-0.496011,-0.117120,1.208346,-0.131037,0.554655,0.505527,1.911632,1.776139,1.782573,1.401236


In [21]:
#study test data
data_test.head()

,AF3,F7,F3,FC5,T7,P7,O1,02,P8,T8,FC6,F4,F8,AF4,index
0,4296.41,4040.51,4253.33,4124.10,4341.54,4618.46,4075.38,4601.03,4183.59,4204.10,4197.44,4268.72,4598.46,4342.56,0
1,4291.28,3994.36,4247.18,4102.56,4328.21,4616.41,4057.44,4622.56,4198.46,4227.69,4190.77,4260.51,4593.33,4337.95,1
2,4299.49,4019.49,4269.74,4116.41,4344.10,4635.38,4067.18,4627.18,4211.28,4233.33,4202.56,4280.51,4596.92,4350.26,2
3,4280.00,4004.62,4263.59,4120.51,4323.59,4603.08,4040.51,4589.23,4174.87,4212.82,4192.82,4271.79,4608.72,4344.10,3
4,4317.44,3968.72,4260.51,4101.54,4341.54,4600.51,4071.28,4607.69,4191.28,4231.28,4199.49,4282.05,4592.31,4372.82,4


In [22]:
#preprocess test data
X_test = data_test.copy()
index = X_test.pop('index')

X_test = pd.DataFrame(feature_scaler.transform(X_test))
X_test.columns = data_test.columns[:-1]

X_test.head()

,AF3,F7,F3,FC5,T7,P7,O1,02,P8,T8,FC6,F4,F8,AF4
0,-0.103077,1.046608,-0.506702,0.072168,-0.004007,-0.088835,0.154276,-0.772271,-0.971812,-1.346811,-0.169859,-0.495161,-0.202948,-0.466549
1,-0.244195,-0.499673,-0.802616,-0.975478,-0.802839,-0.202532,-0.702932,0.398702,-0.136692,-0.146683,-0.443319,-0.913170,-0.358124,-0.590924
2,-0.018351,0.342321,0.282880,-0.301852,0.149407,0.849583,-0.237536,0.649974,0.583297,0.140249,0.040053,0.105122,-0.249531,-0.258808
3,-0.554489,-0.155906,-0.013033,-0.102439,-1.079704,-0.941841,-1.511881,-1.414048,-1.461540,-0.903186,-0.359272,-0.338853,0.107403,-0.425001
4,0.475423,-1.358756,-0.161230,-1.025087,-0.004007,-1.084378,-0.041630,-0.410047,-0.539931,0.035956,-0.085812,0.183531,-0.388978,0.349848


In [23]:
#2 classes -> logistic regression?
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np

model = LogisticRegression()
scores = cross_val_score(model, X_train, y_train)
score = np.mean(scores)
print('accuracy score for model = {}'.format(score))

model.fit(X_train, y_train)
predictions = pd.DataFrame(model.predict_proba(X_test))
predictions.columns=['0', '1']
sample_submission = pd.DataFrame()
sample_submission['index'] = index
sample_submission['label'] = predictions['1']




accuracy score for model = 0.6205


In [24]:
#XGBoost?
from xgboost import XGBRegressor

model = XGBRegressor()
scores = cross_val_score(model, X_train, y_train)
score = np.mean(scores)
print(score)


[00:00:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:00:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:00:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:00:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:00:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.41161151013929764


In [28]:
#Decision Tree?
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
scores = cross_val_score(model, X_train, y_train)
score = np.mean(scores)
print(score)

0.742


In [29]:
#Naive Bayes?
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
scores = cross_val_score(model, X_train, y_train)
score = np.mean(scores)
print(score)

0.6040000000000001


In [31]:
#K
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
scores = cross_val_score(model, X_train, y_train)
score = np.mean(scores)
print(score)

0.866


In [26]:
#RandomForest?
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
scores = cross_val_score(model, X_train, y_train)
score = np.mean(scores)
print(score)


0.5009095288670848


In [27]:
#support vector machines
from sklearn.svm import SVC

clf = SVC(kernel='linear')
scores = cross_val_score(clf, X_train, y_train)
score = np.mean(scores)
print(score)


0.6399999999999999


In [0]:
#All models grouped together
models = dict()
models['dec_tr'] = DecisionTreeClassifier()
models['log_re'] = LogisticRegression()
models['SVM'] = SVC()
models['nai_ba'] = GaussianNB()
models['K_nei'] = KNeighborsClassifier()



In [40]:
#All models are evaluated
for name, model in models.items():
  results = list()
  names = list()
  score = cross_val_score(model, X_train, y_train)
  results.append(score)
  names.append(name)
  print('>%s %.3f (%.3f)' %(name, np.mean(scores), np.std(scores)))

>dec_tr 0.866 (0.012)
>log_re 0.866 (0.012)
>SVM 0.866 (0.012)
>nai_ba 0.866 (0.012)
>K_nei 0.866 (0.012)


Make sure to save your results without the extra Pandas index column that is written by default:

In [0]:
filename = "my_prediction_results.csv"

#make sure to not write the Pandas index column (index=False)
sample_submission.to_csv(filename,index=False)